In [13]:
import pandas as pd
import numpy as np


In [14]:
#load data
#load data
data_df = pd.read_csv("../db/stock_data_with_forward_perf.csv")
print(data_df.columns)
data_df.head()

Index(['name', 'ticker', 'Quarter end', 'Shares', 'Shares split adjusted',
       'Split factor', 'Assets', 'Current Assets', 'Liabilities',
       'Current Liabilities', 'Shareholders equity',
       'Non-controlling interest', 'Preferred equity',
       'Goodwill & intangibles', 'Long-term debt', 'Revenue', 'Earnings',
       'Earnings available for common stockholders', 'EPS basic',
       'EPS diluted', 'Dividend per share', 'Cash from operating activities',
       'Cash from investing activities', 'Cash from financing activities',
       'Cash change during period', 'Cash at end of period',
       'Capital expenditures', 'Price', 'Price high', 'Price low', 'ROE',
       'ROA', 'Book value of equity per share', 'P/B ratio', 'P/E ratio',
       'Cumulative dividends per share', 'Dividend payout ratio',
       'Long-term debt to equity ratio', 'Equity to assets ratio',
       'Net margin', 'Asset turnover', 'Free cash flow per share',
       'Current ratio', 'mktcap_revenue_value', '

,name,ticker,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,...,two_quarter_return,one_year_return,two_year_return,three_year_return,four_year_return,five_year_return,seven_year_return,ten_year_return,twelve_year_return,fifteen_year_return
0,Alcoa Corporation,AA,2018-03-31,482832111,482832111,1,18219000000,5895000000,12937000000,2802000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alcoa Corporation,AA,2017-12-31,482772252,482772252,1,18718000000,6378000000,13794000000,2824000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alcoa Corporation,AA,2017-09-30,481324177,481324177,1,19237000000,6148000000,13276000000,2677000000,...,9.329564,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alcoa Corporation,AA,2017-06-30,441030999,441030999,1,19106000000,6033000000,13353000000,2658000000,...,2.460317,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alcoa Corporation,AA,2017-03-31,440826482,440826482,1,20157000000,6710000000,14662000000,2587000000,...,2.392539,10.948905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#specify the features(measures) and target (label) to be used

import collections

#get features data to be used. Change for comparing for selected parameters
parameters_columns = ['EPS basic','ROE','ROA', 'P/B ratio', 'P/E ratio', 'Dividend payout ratio',
                   'Long-term debt to equity ratio', 'Equity to assets ratio', 'Free cash flow per share',
                   'Current ratio', 'mktcap_revenue_value', 'mktcap_free_cash_flow_value',
                   'mktcap_cash_value', 'cash_oper_gt_earnings_value',
                   'entvalue_earnings_value', 'marketcap_bookvalue_value']

#set target values
target_values = ['no','yes']


#change for selected different investment period
selected_target_column = 'three_year_return'

#set calculated values whether a stock is a buy (1) or not (2). Standard  used is 5% for each year, so 5% for a year, 15% for 3 years and 25 for 5 years
data_df.loc[data_df[selected_target_column] >= 24.0, 'calc_buystock'] = 1
data_df.loc[data_df[selected_target_column] < 24.0, 'calc_buystock'] = 0


#separate data into training and training.
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
#using 1st qtr 2016, change the ending train date according to investment period (selected_target_column). 
#It should that much time period behind the target_start_date to avoid using future data
target_start_date = '2018-01-01'
target_end_date = '2018-03-31'
ending_train_date = '2014-12-31'

test_data_df = data_df.loc[(data_df['Quarter end'] >= target_start_date)&(data_df['Quarter end']<=target_end_date)]
y_test = test_data_df['calc_buystock'].values

print(f"Counts for calculated buy for test data: {collections.Counter(y_test)}")
X_test = test_data_df[parameters_columns].values

train_data_df = data_df.loc[data_df['Quarter end'] <= ending_train_date]
y_train = train_data_df['calc_buystock'].values
X_train = train_data_df[parameters_columns].values
print(f"Counts for calculated buy for train data: {collections.Counter(y_train)}")

Counts for calculated buy for test data: Counter({0.0: 392, 1.0: 56})
Counts for calculated buy for train data: Counter({1.0: 25216, 0.0: 25213})


In [23]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
test_data_df['RF_predicted_buys'] = rf_predictions
print('RF Test Acc: %.3f' % rf.score(X_test, y_test))

RF Test Acc: 0.777


C:\Users\rpaul\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
# KNN model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
test_data_df['KNN_predicted_buys'] = knn_predictions
print('KNN k=13 Test Acc: %.3f' % knn.score(X_test, y_test))

KNN k=13 Test Acc: 0.658


C:\Users\rpaul\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
#Neural networks

from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

#Transform the training and the testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=16))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

dnn_predictions = deep_model.predict_classes(X_test)
test_data_df['DNN_predicted_buys'] = dnn_predictions
#print('DNN Test Acc: %.3f' % deep_model.score(X_test, y_test))

Epoch 1/100
 - 1s - loss: 0.6894 - acc: 0.5435
Epoch 2/100
 - 1s - loss: 0.6704 - acc: 0.5897
Epoch 3/100
 - 1s - loss: 0.6682 - acc: 0.5911
Epoch 4/100
 - 1s - loss: 0.6660 - acc: 0.5911
Epoch 5/100
 - 1s - loss: 0.6642 - acc: 0.5913
Epoch 6/100
 - 1s - loss: 0.6631 - acc: 0.5925
Epoch 7/100
 - 1s - loss: 0.6626 - acc: 0.5922
Epoch 8/100
 - 1s - loss: 0.6621 - acc: 0.5942
Epoch 9/100
 - 1s - loss: 0.6618 - acc: 0.5929
Epoch 10/100
 - 1s - loss: 0.6617 - acc: 0.5943
Epoch 11/100
 - 1s - loss: 0.6613 - acc: 0.5935
Epoch 12/100
 - 1s - loss: 0.6610 - acc: 0.5952
Epoch 13/100
 - 1s - loss: 0.6608 - acc: 0.5940
Epoch 14/100
 - 1s - loss: 0.6604 - acc: 0.5952
Epoch 15/100
 - 1s - loss: 0.6605 - acc: 0.5932
Epoch 16/100
 - 1s - loss: 0.6603 - acc: 0.5951
Epoch 17/100
 - 1s - loss: 0.6602 - acc: 0.5954
Epoch 18/100
 - 1s - loss: 0.6601 - acc: 0.5944
Epoch 19/100
 - 1s - loss: 0.6598 - acc: 0.5940
Epoch 20/100
 - 1s - loss: 0.6602 - acc: 0.5955
Epoch 21/100
 - 1s - loss: 0.6599 - acc: 0.5953
E

C:\Users\rpaul\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# works only for skikit models
from sklearn.ensemble import VotingClassifier
ensbl = VotingClassifier(estimators=[('rf', rf), ('knn', knn)], voting='hard')
ensbl.fit(X_train,y_train)
ensbl_predictions = ensbl.predict(X_test)
test_data_df['Ensemble_predicted_buys'] = ensbl_predictions
print('Ensemble Test Acc: %.3f' % ensbl.score(X_test,y_test))

In [26]:
from statistics import mode

final_pred = np.array([])
for i in range(0,len(X_test)):
    final_pred = np.append(final_pred, mode([rf_predictions[i], knn_predictions[i], dnn_predictions[i]]))
test_data_df['Ensemble_predicted_buys'] = final_pred

C:\Users\rpaul\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
test_data_df.to_csv('stock_predictions_1qtr2018.csv')

In [ ]:
test_data_df.head()